# Assignment 2: Integer Linear Programming

**Module**: COMP30930 Optimisation, Autumn 2021<br/>
**Student**: Rajit Banerjee, 18202817

In [1]:
using JuMP
using Xpress

┌ Info: Xpress: Found license file /Applications/FICO Xpress/xpressmp/bin/xpauth.xpr
└ @ Xpress /Users/rajitbanerjee/.julia/packages/Xpress/eJoYN/src/license.jl:44
┌ Info: Xpress: Development license detected.
└ @ Xpress /Users/rajitbanerjee/.julia/packages/Xpress/eJoYN/src/license.jl:89


## Question 1: Modified Knapsack Problem

- Classical knapsack problem: fill your knapsack with one or more of the six items below, while maximising the utility and allowing a weight of 15 Kg. 
- Additional constraint: at least 3 items must be picked.

|  | C1 | C2 | C3 | C4 | C5 | C6 |
| --- | --- | --- | --- | --- | --- | --- |
| Utility | 4 | 2 | 1 | 7 | 3 | 6 |
| Weight (Kg) | 5 | 8 | 8 | 6 | 1 | 5 |

Let the $i^{th}$ item have a utility $u_i$ units and weight $w_i$ Kg. The objective is to select a subset with maximum utility, such that the summation of the weights of the selected items is less than or equal to 15 Kg. Moreover, the summation of indicator variables $x_i$ essentially counts the number of items in the backpack, which must be at least 3.
 
Considering a set of binary decision variables $x_i$ that receive value 1 if the $i^{th}$ item is selected, and 0 if it is not selected, the resulting mathematical programming formulation (Integer Linear Program or ILP) is:

$$
\begin{align}
Max.&\; \sum_{i \in I}u_i.x_i \\
s.t.&\; \sum_{i \in I}w_i.x_i \le 15 \\
&\sum_{i \in I}x_i \ge 3 \\
x_i&\in\{0,1\}\;\; \forall i\in I
\end{align}
$$


In [2]:
utility = [4, 2, 1, 7, 3, 6]
weight = [5, 8, 8, 6, 1, 5]
N = 6
W = 15
min_items = 3;

In [3]:
# helper to optimise any given model and show results
function solve(model, x)
    optimize!(model)
    if termination_status(model) == MOI.OPTIMAL
        @show value.(x)
        @show objective_value(model)
    elseif termination_status(model) == MOI.INFEASIBLE
        println("The model is infeasible!")
    else
        println("The model didn't solve properly for some other reason.")
    end
end;

In [4]:
model = Model(Xpress.Optimizer)

# Decision variables: x[i] = 1 if item i is included, else 0
@variable(model, x[1:N], Bin)

# Constraints
@constraint(model, weight' * x ≤ W)
@constraint(model, sum(x) ≥ min_items)

# Maximise utility
@objective(model, Max, utility' * x)
solve(model, x);

value.(x) = [0.0, -0.0, -0.0, 1.0, 1.0, 1.0]
objective_value(model) = 16.0


The solution suggests that items $C_4, C_5, C_6$ should be selected to obtain the maximum utility.

## Question 2: Branch-and-Bound

Original binary integer problem:

$$
\begin{align}
Max.&\; 12x_1 + x_2 + 10x_3 + 2x_4 \\
s.t.&\; 16x_1 + 2x_2 + 10x_3 + 8x_4 \le 18 \\
x_i &\in \{0, 1\} \;for\; i = 1, ..., 4 \\
\end{align}
$$

The above ILP can be solved using the branch-and-bound algorithm by systematically fixing values of $x_1, x_2, x_3, x_4$, to partition the entire set of feasible solutions into smaller and smaller subsets. The subproblems thus obtained can be solved using LP (Linear Program) relaxations, and fathoming steps can be applied to prune branches until we find an optimal integer-valued solution, if it exists.

**Subproblem 0: Fixing $x_1 = 0$**

$$
\begin{align}
Max.&\; 12x_1 + x_2 + 10x_3 + 2x_4 \\
s.t.&\; 16x_1 + 2x_2 + 10x_3 + 8x_4 \le 18 \\
0 &\le x_i \le 1 \;for\; i = 2, 3, 4 \\
x_1 &= 0
\end{align}
$$

In [5]:
# Subproblem 0
model = Model(Xpress.Optimizer)
@variable(model, 0 ≤ x[1:4] ≤ 1)
@objective(model, Max, 12x[1] + x[2] + 10x[3] + 2x[4])

@constraint(model, 16x[1] + 2x[2] + 10x[3] + 8x[4] ≤ 18)
@constraint(model, x[1] == 0)

solve(model, x);

value.(x) = [0.0, 1.0, 1.0, 0.75]
objective_value(model) = 12.5


Since this a maximisation problem, we have achieved an upper bound = 12.5. We will keep this branch open for further exploration later.

**Subproblem 1: Fixing $x_1 = 1$**

$$
\begin{align}
Max.&\; 12x_1 + x_2 + 10x_3 + 2x_4 \\
s.t.&\; 16x_1 + 2x_2 + 10x_3 + 8x_4 \le 18 \\
0 &\le x_i \le 1 \;for\; i = 2, 3, 4 \\
x_1 &= 1
\end{align}
$$

In [6]:
# Subproblem 1
model = Model(Xpress.Optimizer)
@variable(model, 0 ≤ x[1:4] ≤ 1)
@objective(model, Max, 12x[1] + x[2] + 10x[3] + 2x[4])

@constraint(model, 16x[1] + 2x[2] + 10x[3] + 8x[4] ≤ 18)
@constraint(model, x[1] == 1)

solve(model, x);

value.(x) = [1.0, 0.0, 0.2, 0.0]
objective_value(model) = 14.0


Subproblem 1 gives a better upper bound = 14, hence we can explore this branch further.

**Subproblem 1-0: Fixing $x_1 = 1, x_2 = 0$**

$$
\begin{align}
Max.&\; 12x_1 + x_2 + 10x_3 + 2x_4 \\
s.t.&\; 16x_1 + 2x_2 + 10x_3 + 8x_4 \le 18 \\
0 &\le x_i \le 1 \;for\; i = 3, 4 \\
x_1 &= 1, x_2 = 0
\end{align}
$$

In [7]:
# Subproblem 1-0
model = Model(Xpress.Optimizer)
@variable(model, 0 ≤ x[1:4] ≤ 1)
@objective(model, Max, 12x[1] + x[2] + 10x[3] + 2x[4])

@constraint(model, 16x[1] + 2x[2] + 10x[3] + 8x[4] ≤ 18)
@constraint(model, x[1] == 1)
@constraint(model, x[2] == 0)

solve(model, x);

value.(x) = [1.0, 0.0, 0.2, 0.0]
objective_value(model) = 14.0


Best upper bound stays at 14.

**Subproblem 1-1: Fixing $x_1 = 1, x_2 = 1$**

$$
\begin{align}
Max.&\; 12x_1 + x_2 + 10x_3 + 2x_4 \\
s.t.&\; 16x_1 + 2x_2 + 10x_3 + 8x_4 \le 18 \\
0 &\le x_i \le 1 \;for\; i = 3, 4 \\
x_1 &= 1, x_2 = 1
\end{align}
$$

In [8]:
# Subproblem 1-1
model = Model(Xpress.Optimizer)
@variable(model, 0 ≤ x[1:4] ≤ 1)
@objective(model, Max, 12x[1] + x[2] + 10x[3] + 2x[4])

@constraint(model, 16x[1] + 2x[2] + 10x[3] + 8x[4] ≤ 18)
@constraint(model, x[1] == 1)
@constraint(model, x[2] == 1)

solve(model, x);

value.(x) = [1.0, 1.0, 0.0, 0.0]
objective_value(model) = 13.0


We have obtained an integer-valued solution with z* = 13, allowing us to fathom the current branch with $x_1 = 1, x_2 = 1$. Moreover, the branch with $x_1 = 0$ (objective value = 12.5) can also be fathomed, since 12.5 < z*.

The only branch still open for exploration starts with $x_1 = 1, x_2 = 0$, with an upper bound of 14.

**Subproblem 1-0-0: Fixing $x_1 = 1, x_2 = 0, x_3 = 0$**

$$
\begin{align}
Max.&\; 12x_1 + x_2 + 10x_3 + 2x_4 \\
s.t.&\; 16x_1 + 2x_2 + 10x_3 + 8x_4 \le 18 \\
0 &\le x_4 \le 1 \\
x_1 &= 1, x_2 = 0, x_3 = 0
\end{align}
$$

In [9]:
# Subproblem 1-0-0
model = Model(Xpress.Optimizer)
@variable(model, 0 ≤ x[1:4] ≤ 1)
@objective(model, Max, 12x[1] + x[2] + 10x[3] + 2x[4])

@constraint(model, 16x[1] + 2x[2] + 10x[3] + 8x[4] ≤ 18)
@constraint(model, x[1] == 1)
@constraint(model, x[2] == 0)
@constraint(model, x[3] == 0)

solve(model, x);

value.(x) = [1.0, 0.0, 0.0, 0.25]
objective_value(model) = 12.5


Fathom branch with objective value 12.5 < z*.

**Subproblem 1-0-1: Fixing $x_1 = 1, x_2 = 0, x_3 = 1$**

$$
\begin{align}
Max.&\; 12x_1 + x_2 + 10x_3 + 2x_4 \\
s.t.&\; 16x_1 + 2x_2 + 10x_3 + 8x_4 \le 18 \\
0 &\le x_4 \le 1 \\
x_1 &= 1, x_2 = 0, x_3 = 1
\end{align}
$$

In [10]:
# Subproblem 1-0-1
model = Model(Xpress.Optimizer)
@variable(model, 0 ≤ x[1:4] ≤ 1)
@objective(model, Max, 12x[1] + x[2] + 10x[3] + 2x[4])

@constraint(model, 16x[1] + 2x[2] + 10x[3] + 8x[4] ≤ 18)
@constraint(model, x[1] == 1)
@constraint(model, x[2] == 0)
@constraint(model, x[3] == 1)

solve(model, x);

The model is infeasible!


Fathom infeasible subproblem.

Hence, all possible branches have now been explored, and the optimal integer-valued solution is as follows:

```
value.(x) = [1.0, 1.0, 0.0, 0.0]
objective_value(model) = 13.0
```

The above solution can be verified by solving the ILP directly below:

In [11]:
model = Model(Xpress.Optimizer)
@variable(model, x[1:4], Bin)
@objective(model, Max, 12x[1] + x[2] + 10x[3] + 2x[4])

@constraint(model, 16x[1] + 2x[2] + 10x[3] + 8x[4] ≤ 18)

solve(model, x);

value.(x) = [1.0, 1.0, -0.0, -0.0]
objective_value(model) = 13.0


Verified. Branch-and-bound successfully found an optimal solution for the given ILP.

## Question 3: Vertex Cover

Given a graph $G = (V, E)$ (vertices and edges), the minimum vertex cover problem can be formulated as a minimisation of the number of vertices selected, such that every edge in the graph is covered (at least one vertex of an edge must be in the cover set).

Considering a set of binary decision variables $x_i$ that receive value 1 if vertex $i$  is selected, and 0 if it is not selected, the resulting mathematical programming formulation (Integer Linear Program or ILP) is:

$$
\begin{align}
Min.&\; \sum_{i \in V} x_i \\
s.t.&\; x_u + x_v \ge 1 \;\;\forall \{u, v\} \in E \\
x_i &\in \{0, 1\} \;\;\forall i \in V
\end{align}
$$

Given an undirected, unweighted graph:
$$
V = \{1, 2, 3, 4, 5, 6\} \\
E = \{\{1, 2\}, \{1, 3\}, \{2, 3\}, \{3, 4\}, \{4, 5\}, \{4, 6\}, \{5, 6\}\}
$$



In [12]:
# Graph G = (V, E)
V = [1, 2, 3, 4, 5, 6]
E = [
    (1, 2),
    (1, 3),
    (2, 3),
    (3, 4),
    (4, 5),
    (4, 6),
    (5, 6)
]
N = 6;

In [13]:
model = Model(Xpress.Optimizer)

# Decision variables: x[i] = 1 if vertex i is included, else 0
@variable(model, x[1:N], Bin)
@objective(model, Min, sum(x))

# Constraints: every edge of G must be covered
for (u, v) in E
    # At least one of x[u] or x[v] should be selected
    @constraint(model, x[u] + x[v] ≥ 1)
end

solve(model, x);

value.(x) = [1.0, -0.0, 1.0, -0.0, 1.0, 1.0]
objective_value(model) = 4.0


The solution suggests that the **minimum vertex cover** $S_1 = \{1, 3, 5, 6\}$.

Next, we can consider the LP relaxation of the ILP formulation above.

$$
\begin{align}
Min.&\; \sum_{i \in V} x_i \\
s.t.&\; x_u + x_v \ge 1 \;\;\forall \{u, v\} \in E \\
0 &\le x_i \le 1 \;\;\forall i \in V
\end{align}
$$

In [14]:
model = Model(Xpress.Optimizer)
@variable(model, 0 ≤ x[1:N] ≤ 1)
@objective(model, Min, sum(x))

for (u, v) in E
    @constraint(model, x[u] + x[v] ≥ 1)
end

solve(model, x);

value.(x) = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
objective_value(model) = 3.0


From the super-optimal fractional solution $S_2$ of the LP relaxation above, we can obtain a legal integral solution: the set of vertices, $S_3 = \{1, 2, 3, 4, 5, 6\}$. 

All 6 vertices are included in the set, since each of the decision variables in $S_2$ has a value $ \ge 0.5$.

Cardinality of $S_3$ is 6, whereas that of $S_1$ is only 4. This is significant because the LP relaxation of any minimum vertex cover problem is a 2-approximation problem, i.e. the approximation yields a result that is at most twice (in terms of set cardinality in this problem) of the optimal solution.